# 3章Policy Function Iteration

### 0. 準備

In [4]:
# ライブラリのインポート
import numpy as np
from collections.abc import Callable
from scipy.optimize import fmin

In [5]:
# 使用する関数の定義
def output(k, A, alpha):
    return A * (k ** alpha)
def wealth(k: float, 
            A:float, 
            alpha: float, 
            delta: float,
            output: Callable[[float, float, float], float]):
    return output(k, A, alpha) + (1 - delta) * k
def utility(c: float, gamma: float):
    if gamma == 1:
        return np.log(c)
    else: 
        return (c ** (1 - gamma)) / (1 - gamma)

### 1. グリッド生成

In [6]:
nk = 21
k_grid = np.linspace(0.025, 0.5, 21)
print(k_grid)

[0.025   0.04875 0.0725  0.09625 0.12    0.14375 0.1675  0.19125 0.215
 0.23875 0.2625  0.28625 0.31    0.33375 0.3575  0.38125 0.405   0.42875
 0.4525  0.47625 0.5    ]


### 2. カリブレーションしたパラメータの値を設定

In [7]:
beta = 0.96
gamma = 1.0
alpha = 0.4
delta = 1.0

### 3. 収束の基準を与える

In [8]:
epsilon = 1e-5

### 4. 確率変数 $A = \{A_{good}, A_{bad}\}$を定義

In [9]:
A = np.array([1.01, 0.99])

### 5. 遷移確率行列 $P$を定義

In [10]:
P = np.array([[0.875, 0.125], [0.125, 0.875]])

### 6. 政策関数の初期値をguess

- 今回は富の関数の半分の値とする

In [11]:
kprime_grid = np.zeros((k_grid.size, A.size))
for i, k in enumerate(k_grid):
    for l, a in enumerate(A):
        kprime_grid[i, l] = wealth(k, a, alpha, delta, output) /2

### 7. 当て推量した政策関数を用いて価値関数を計算する

In [14]:
# a. 初期の価値関数をguess
vf_old = np.zeros((k_grid.size, A.size))
diff = 1.0 + epsilon
while diff > epsilon:
    for loop in range(1000) :# 7b. 新たな価値観数を得る
        vf_new = np.zeros((k_grid.size, A.size))
        for i, K in enumerate(k_grid):
            for l, a in enumerate(A):
                for k, aprime in enumerate(A): # 次期の技術水準の添字 k
                    vf_new[i, l] += P[l ,k] * (utility(wealth(K, a, alpha, delta, output) 
                                                - kprime_grid[i, l], gamma) + 
                                        beta * vf_old[i, k])
        if np.max(np.abs(vf_old - vf_new)) < epsilon:
            break
        vf_old = vf_new.copy()
    
    # 8 得られた新しい価値関数を用いて政策関数を求める
    def vf_interp(k_grid, A, vf_grid): # 価値関数の補間する関数
        def vf(k, a):
            A_index = np.where(A == a)[0][0]
            cheb_fit = np.polynomial.Chebyshev.fit(k_grid, vf_grid[:, A_index], deg = 8)
            return cheb_fit(k)
        return vf
    vf = vf_interp(k_grid, A, vf_new) # 価値関数の補間
    
    def rhs_bellman(kprime_star, k ,a): # ベルマン方程式の右辺
        rhs = utility(wealth(k, a, alpha, delta, output) - kprime_star,
                    gamma) + beta * vf(kprime_star, a)
        return - rhs
    
    for i, k in enumerate(k_grid): # 右辺を最小化するkprime_starを求める
        for l, a in enumerate(A):
            kprime_grid[i, l] = fmin(rhs_bellman, k_grid[0], args = (k, a), disp = 0)
    
    vf_new = np.zeros((k_grid.size, A.size)) # 得られた政策関数を用いて新しい価値関数を求める
    for i, K in enumerate(k_grid):
        for l, a in enumerate(A):
            for k, aprime in enumerate(A): # 次期の技術水準の添字 k
                vf_new[i, l] += P[l ,k]*(utility(wealth(K, a, alpha, delta, output) 
                                            - kprime_grid[i, l], gamma) + 
                                    beta * vf_old[i, k])

    diff = np.max(np.abs(vf_old - vf_new)) # 収束の判定

/var/folders/y2/hwl07kg14b31nb5s126cjc8m0000gn/T/ipykernel_89113/146179979.py:12: RuntimeWarning: invalid value encountered in log
  return np.log(c)
/var/folders/y2/hwl07kg14b31nb5s126cjc8m0000gn/T/ipykernel_89113/4215163994.py:33: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  kprime_grid[i, l] = fmin(rhs_bellman, k_grid[0], args = (k, a), disp = 0)


### p78 図3.6の価値関数と政策関数をプロット

In [17]:
print(kprime_grid)

[[-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]
 [-1.58456325e+27 -1.58456325e+27]]
